# Prompting Capabilities 

When you first start using Mistral models, your first interaction will revolve around prompts. The art of crafting effective prompts is essential for generating desirable responses from Mistral models or other LLMs. This guide will walk you through example prompts showing four different prompting capabilities. 

- Classification 
- Summarization 
- Personalization
- Evaluation


In [1]:
! pip install mistralai

In [1]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os, dotenv

dotenv.load_dotenv()

assert "GITHUB_TOKEN" in os.environ, "Please set the GITHUB_TOKEN environment variable."
github_token = os.environ["GITHUB_TOKEN"]

# We can use some defaults for the other two variables
endpoint = "https://models.inference.ai.azure.com"
model_name = "mistral-large"


In [2]:
def run_mistral(user_message, model=model_name):
    client = MistralClient(api_key=github_token, endpoint=endpoint)
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

## Classification
Mistral models can easily categorize text into distinct classes. In this example prompt, we can define a list of predefined categories and ask Mistral models to classify user inquiry. 



In [3]:
def user_message(inquiry):
    user_message = (
        f"""
        You are a bank customer service bot. Your task is to assess customer intent 
        and categorize customer inquiry after <<<>>> into one of the following predefined categories:
        
        card arrival
        change pin
        exchange rate
        country support 
        cancel transfer
        charge dispute
        
        If the text doesn't fit into any of the above categories, classify it as:
        customer service
        
        You will only respond with the predefined category. Do not include the word "Category". Do not provide explanations or notes. 
        
        ####
        Here are some examples:
        
        Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card as expected. Could you please provide information about the tracking process for my card, or confirm if there are any indicators to identify if the card has been lost during delivery?
        Category: card arrival
        Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees for foreign transactions.
        Category: exchange rate 
        Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, and would appreciate any information regarding compatibility and functionality in these regions.
        Category: country support
        Inquiry: Can I get help starting my computer? I am having difficulty starting my computer, and would appreciate your expertise in helping me troubleshoot the issue. 
        Category: customer service
        ###
    
        <<<
        Inquiry: {inquiry}
        >>>
        """
    )
    return user_message

### Strategies we used: 

- **Few shot learning**: Few-shot learning or in-context learning is when we give a few examples in the prompts, and the LLM can generate corresponding output based on the example demonstrations. Few-shot learning can often improve model performance especially when the task is difficult or when we want the model to respond in a specific manner. 
- **Delimiter**: Delimiters like ### <<< >>> specify the boundary between different sections of the text. In our example, we used ### to indicate examples and <<<>>> to indicate customer inquiry. 
- **Role playing**: Providing LLM a role (e.g., "You are a bank customer service bot.") adds personal context to the model and often leads to better performance. 

In [4]:
print(run_mistral(user_message(
    "I am inquiring about the availability of your cards in the EU, as I am a resident of France and am interested in using your cards. "
)))

country support


In [5]:
print(run_mistral(user_message("What's the weather today?")))

customer service


## Summarization

Summarization is a common task for LLMs due to their natural language understanding and generation capabilities. Here is an example prompt we can use to generate interesting questions about an essay and summarize the essay.


In [6]:
import requests
response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
# need to limit the size to meet the token limit of the GitHub Models free service
essay = response.text[:20000]

In [7]:
message = f"""
You are a commentator. Your task is to write a report on an essay. 
When presented with the essay, come up with interesting questions to ask, and answer each question. 
Afterward, combine all the information and write a report in the markdown format. 

# Essay: 
{essay}

# Instructions: 
## Summarize:
In clear and concise language, summarize the key points and themes presented in the essay.

## Interesting Questions: 
Generate three distinct and thought-provoking questions that can be asked about the content of the essay. For each question:
- After "Q: ", describe the problem 
- After "A: ", provide a detailed explanation of the problem addressed in the question.
- Enclose the ultimate answer in <>.

## Write a report 
Using the essay summary and the answers to the interesting questions, create a comprehensive report in Markdown format. 
"""

In [8]:
print(run_mistral(message))

# Report

## Summary

The essay is a personal account of the author's journey through various interests and careers, from writing and programming to philosophy, artificial intelligence, and finally, painting. The author begins by discussing their early attempts at writing short stories and programming on an IBM 1401 during their school days. They express their initial fascination with philosophy, which they later found disappointing in college. They then delve into their exploration of AI, which they eventually realized was not as promising as they had initially thought. This realization led them to focus on Lisp, a programming language they found interesting in its own right. Despite their love for building things, the author felt dissatisfied with the transient nature of systems work, leading them to pursue painting as a means of creating something lasting.

## Interesting Questions

**Q1: How did the author's perception of philosophy change from high school to college?**

*A1:* In h

## Strategies we used: 

- **Step-by-step instructions**: This strategy is inspired by the chain-of-thought prompting that enables LLMs to use a series of intermediate reasoning steps to tackle complex tasks. It's often easier to solve complex problems when we decompose them into simpler and small steps and it's easier for us to debug and inspect the model behavior.  In our example, we break down the task into three steps: summarize, generate interesting questions, and write a report. This helps the language to think in each step and generate a more comprehensive final report. 
- **Example generation**: We can ask LLMs to automatically guide the reasoning and understanding process by generating examples with the explanations and steps. In this example, we ask the LLM to generate three questions and provide detailed explanations for each question. 
- **Output formatting**: We can ask LLMs to output in a certain format by directly asking "write a report in the Markdown format".



## Personlization

LLMs excel at personalization tasks as they can deliver content that aligns closely with individual users. In this example, we create personalized email responses to address customer questions. 

In [9]:
email = """
Dear mortgage lender, 

What's your 30-year fixed-rate APR, how is it compared to the 15-year fixed rate?

Regards,
Anna
"""

In [10]:
message = f"""

You are a mortgage lender customer service bot, and your task is to create personalized email responses to address customer questions.
Answer the customer's inquiry using the provided facts below. Ensure that your response is clear, concise, and 
directly addresses the customer's question. Address the customer in a friendly and professional manner. Sign the email with 
"Lender Customer Support."   


      
# Facts
30-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%

# Email
{email}
"""

In [11]:
print(run_mistral(message))

Subject: Information on Our 30-Year and 15-Year Fixed-Rate Mortgages

Dear Anna,

Thank you for reaching out to us with your inquiry about our fixed-rate mortgage options. We are committed to providing you with the information you need to make an informed decision.

Our current 30-year fixed-rate mortgage has an Annual Percentage Rate (APR) of 6.484%. In comparison, the 15-year fixed-rate mortgage has a lower APR of 5.848%. The difference in APR between the two options reflects the longer duration and associated risks of the 30-year fixed-rate mortgage. However, choosing a shorter term can result in higher monthly payments, even though you will pay less in interest over the life of the loan.

We encourage you to consider your financial goals and budget when selecting the mortgage term that best suits your needs. Our team is available to provide further assistance and discuss your options in more detail.

Please feel free to reply to this email or call us directly if you have any additi

### Strategies we used: 
- Providing facts: Incorporating facts into prompts can be useful for developing customer support bots. It’s important to use clear and concise language when presenting these facts. This can help the LLM to provide accurate and quick responses to customer queries.

## Evaluation

There are many ways to evaluate LLM outputs. Here are three approaches for your reference: include a confidence score, introduce an evaluation step, or employ another LLM for evaluation.



## Include a confidence score
We can include a confidence score along with the generated output in the prompt. 

In [12]:
def run_mistral(user_message, model=model_name):
    client = MistralClient(api_key=github_token, endpoint=endpoint)
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages,
        temperature=1
    )
    return (chat_response.choices[0].message.content)

In [13]:
message = f"""
You are a summarization system that can provide summaries with associated confidence scores.
In clear and concise language, provide three short summaries of the following essay, along with their confidence scores. 
You will only respond with a JSON object with the key Summary and Confidence. Do not provide explanations. 

# Essay: 
{essay}

"""

In [14]:
print(run_mistral(message))

{
"Summary 1": "The author recounts their early experiences with writing and programming, starting with writing short stories and moving on to programming on an IBM 1401 in 9th grade. They found the 1401 limited and were excited by the advent of microcomputers, eventually convincing their father to buy a TRS-80.",
"Confidence 1": "0.95"
},
{
"Summary 2": "The author attended college planning to study philosophy, but found it boring and switched to AI after being inspired by a novel and a PBS documentary. They started learning Lisp and reversed-engineered SHRDLU for their undergraduate thesis.",
"Confidence 2": "0.9"
},
{
"Summary 3": "During grad school, the author realized that the approach to AI at the time was a 'hoax' and decided to focus on Lisp instead, even writing a book about Lisp hacking. They wanted to build things that would last and considered art as a possible career path.",
"Confidence 3": "0.85"
}


### Strategies we used: 
- JSON output: For facilitating downstream tasks, JSON format output is frequently preferred. We can specify in the prompt that “You will only respond with a JSON object with the key Summary and Confidence.” Specifying these keys within the JSON object is beneficial for clarity and consistency.
- Higher Temperature: In this example, we increase the temperature score to encourage the model to be more creative and output three generated summaries that are different from each other.  


## Introduce an evaluation step 
We can also add a second step in the prompt for evaluation. 


In [15]:
message = f"""
You are given an essay text and need to provide summaries and evaluate them.

# Essay: 
{essay}

Step 1: In this step, provide three short summaries of the given essay. Each summary should be clear, concise, and capture the key points of the speech. Aim for around 2-3 sentences for each summary.
Step 2: Evaluate the three summaries from Step 1 and rate which one you believe is the best. Explain your choice by pointing out specific reasons such as clarity, completeness, and relevance to the speech content.


"""
print(run_mistral(message))

Step 1:

Summary 1:
The essay recounts the author's journey from writing and programming in their youth to studying philosophy and AI in college, and their eventual disillusionment with AI. The author then turns to Lisp and writes a book about it, while also nurturing an interest in painting.

Summary 2:
The author narrates their early experiences with writing, programming, and studying philosophy and AI in college. They realize the limitations of AI and shift their focus to Lisp, writing a book called "On Lisp". The author also develops a keen interest in painting and decides to pursue it professionally.

Summary 3:
The author shares their journey from being a young writer and programmer to studying philosophy and AI in college. They become disenchanted with AI and turn to Lisp, writing a book on the subject. Meanwhile, they discover a passion for painting and decide to make it their career.

Step 2:

The best summary among the three is Summary 2. This summary effectively captures the

## Employ another LLM for evaluation
In production systems, it is common to employ another LLM for evaluation so that the evaluation step can be separate from the generation step.  

- Step 1: use the first LLM to generate three summaries 



In [16]:
message = f"""
Provide three short summaries of the given essay. Each summary should be clear, concise, and capture the key points of the essay.
Aim for around 2-3 sentences for each summary.

# essay: 
{essay}

"""
summaries = run_mistral(message)

In [17]:
print(summaries)

1. The author recounts their early experiences with writing and programming before college. They wrote short stories, which they now realize were not very good, and tried programming on an IBM 1401, but found it limited. It wasn't until microcomputers became available that they really started programming, writing games and other simple programs.

2. The author initially planned to study philosophy in college, but found it boring and switched to AI after being inspired by a novel and a documentary. They started learning Lisp and working on AI projects, but eventually realized that the type of AI they were working on was not going to lead to true understanding of natural language.

3. After becoming disillusioned with AI, the author decided to focus on Lisp and write a book about it. They also became interested in painting and took art classes at Harvard. They eventually graduated from a PhD program in computer science and went to art school in Florence, where they painted still lives an

- Step 2: use another LLM to rate the generated summaries


In [18]:
message = f"""
You are given an essay and three summaries of the essay. Evaluate the three summaries and rate which one you believe is the best. 
Explain your choice by pointing out specific reasons such as clarity, completeness, and relevance to the essay content.

# Essay: 
{essay}

# Summaries 
{summaries}

"""
print(run_mistral(message))

Summary 3 is the best summary of the essay. It provides a clear and complete overview of the author's experiences with writing, programming, and AI, as well as their eventual shift towards focusing on Lisp and painting. It highlights the key turning points in the author's career, such as their realization that the type of AI they were working on was not going to be successful, and their decision to write a book about Lisp and pursue painting. The summary also includes specific details about the author's experiences, such as their interest in still life painting and their observations about the low end eating the high end in technology. Overall, Summary 3 effectively captures the main points of the essay and provides a well-rounded and relevant summary of the author's experiences.


### Strategies we used: 
- **LLM chaining**: In this example, we chain two LLMs in a sequence, where the output from the first LLM serves as the input for the second LLM. The method of chaining LLMs can be adapted to suit your specific use cases. For instance, you might choose to employ three LLMs in a chain, where the output of two LLMs is funneled into the third LLM. While LLM chaining offers flexibility, it's important to consider that it may result in additional API calls and potentially increased costs.


